In [ ]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import pywt
import os
import re
from scipy.signal import butter, filtfilt

def calculate_psd_features(file_path):
    with open(file_path, "r") as file:
        signal_data = file.readlines()
        signal_data = [float(data_point) for data_point in signal_data]

        # Apply bandpass filter
        lowcut = 0.5
        highcut = 20
        signal_data = apply_bandpass_filter(signal_data, lowcut, highcut)

        # Calculate PSD
        f, psd = signal.welch(signal_data, fs=50, nperseg=1024)

        # Extract features from PSD
         
        var_psd = np.var(psd)
        mean_psd = np.mean(psd)
        std_psd = np.std(psd)
        energy_psd = np.sum(psd)

        return   var_psd, mean_psd, std_psd, energy_psd

def apply_bandpass_filter(signal_data, lowcut, highcut, fs=50, order=5 ):
    # Design the bandpass filter
    nyquist_freq = 0.5 * fs
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = butter(4, [low, high], btype="band")

    # Apply the filter to the signal
    filtered_signal = filtfilt(b, a, signal_data)
    return filtered_signal

directory = "/kaggle/input/data-eog-last-v/3-class"
files = os.listdir(directory)

# Sort the file names in ascending order
sorted_files = sorted(files)

psd_features_list = []

for file_name in sorted_files:
    if file_name.endswith(".txt"):
        file_path = os.path.join(directory, file_name)
        var_psd, mean_psd, std_psd, energy_psd = calculate_psd_features(file_path)
        std_psd=round(std_psd, 3)
         
        var_psd=round(var_psd, 3)
        mean_psd=round(mean_psd, 3)
        energy_psd=round(energy_psd, 3)
        label = re.split(r"\d", file_name, 1)[0]
        psd_features_list.append(( var_psd, mean_psd, std_psd, energy_psd, label))
print("done") 

In [ ]:

features_psd = []

for i in range(0, 200, 2):
    row = psd_features_list[i] + psd_features_list[i+1]
    features_psd.append(row)
features_psd = [row[:4] + row[5:] for row in features_psd]
 
print (features_psd)

In [ ]:
import pandas as pd

 
df = pd.DataFrame(features_psd, columns=[  "var_h","mean_h","S_D_h","energy_h","var_v" , "mean_v","S_D_v","energy_v", "label_y"])

 
df.to_excel("Features_of_psd.xlsx", index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
data = pd.read_excel('/kaggle/working/Features_of_psd.xlsx')

# Split features and target variable
X = data.drop('label_y', axis=1)
 
y = data['label_y']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize SVM model
svm_model = SVC(random_state=12)

# Define hyperparameters grid for tuning
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': [0.1, 0.01, 0.001],  # Kernel coefficient for 'rbf', 'poly', 'sigmoid'
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Initialize SVM model with best hyperparameters
best_svm_model = SVC(**best_params, random_state=12)

# Train the model with the best hyperparameters
best_svm_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = best_svm_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
